# irp-dbk24 - "Optimising Demand Response Strategies for Carbon-Intelligent Electricity Use"

# Developing Optimisation Models

https://www.ceicdata.com/en/india/electricity-consumption-utilities/electricity-consumption-utilities-delhi

India Delhi Annual consumption (2023) 34,107 GWh
--> Daily consumption (average) = 34,107 GWh / 365 = 93.4 GWh
--> Average Hourly Consumption = 93.4 GWh / 24 = 3.89 GWh
--> Average Half-Hourly Consumption = 3.89 GWh / 2 = 1.945 GWh


**BEHAVIORAL CONSTRAINTS:**

1. Maximum Moves Per Customer Per Day
    * Constraint: moves_per_customer_per_day ≤ X (start with X = 1)
    * Justification: I'm thinking that you can't ask a customer to change their routine too much, so we need to limit the number of moves that we would request in a single day. This minimises disruption to routine and makes it easier to communicate and for households to act on.
    * Parameters Provided: X , timezone, day_boundaries (defaults to hours 0-23 inclusive), week boundaries (defaults to Monday-Sunday, but could also be ISO week)

2. Maximum Moves Per Customer Per Week
    * Constraint: moves_per_customer_per_week ≤ Y (start with Y = 3)
    * Justification: Again, I'm thinking that you cannot ask a customer to change their routine too much, so we need to limit the number of moves that we would request in a single week. Prevents “alert fatigue” where customers stop responding if asked too often.
    * Parameters Provided: Y, timezone, week boundaries (defaults to Monday-Sunday, but could also be ISO week)

3. Max Shift Window
    * Constraint: |t_original - t_shifted| ≤ H  (start with H=2 hours)
    * Justification:  It is unreasonable to ask  customer to move all of their electricity usage to the middle of the night. There would likely be more success in requesting moves that are easily achievable. Therefore we set a constraint saying that power cannot be shifted more than H hours from its original usage time
    * Parameters Provided: H_hours, slot_length=30min, inclusive=True

4. Preserve Peak-Hour Comfort:
    * Constraint: percentage_usage_reduced_in_peak_hours ≤ Z * customers usage in that peak hour (start with Z = 30%)
    * Justification: This constraint is in place to ensure that we do not overly burden customers during peak usage times. By limiting the percentage of usage that can be shifted in a peak hour, we aim to maintain a level of comfort and convenience for customers during critical periods.
    * Parameters Provided: Z_percent, scope={per_customer|per_city}, peak_hours={city{Delhi, Mumbai}, day{Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday}, hour_slots(up to 6)}


**PRACTICAL TECHNICAL CONSTRAINTS**

1. Regional Maximum Shift Percentage
    * Constraint: daily_total_shifted_power ≤ P% x regional_total_daily_average_load (start with p = 10%)
    * Justification: The total amount of power moved through this optimisation should not be too much, otherwise it may be too much change for the grid to handle. As a starting point, I might recommend saying that the total amount of power can't be more than 10% of the total region's daily average load.
    * Note - I have data for Delhi and Mumbai - > separated by city and so I can provide those statistics for the average daily load (based on external government statistics)
    * Parameters Provided: P, regional_total_daily_average_load (kWh), city

2. Household Minimum Usage per Slot:
    * Constraint: shifted_usage_t ≥ max(min_baseline(customer,t), R% * robust_max(customer,t))
    * Justification: When recommending a shifting of power, the power remaining cannot go below X. In other words - minimum electricity usage for that household must be maintained.
    * Implementation Note: The minimum baseline of a customer's usage is calculated, as well as the robust maxiumum (excludes extreme outliers). The maximum of this minimum baseline and 10% of the robust maxiumum is taken and set as the boundary for minimum usage per slot.
    * Note - This constraint is provided assuming the half-hourly usage.
    * Parameters Provided: baseline_period={year|month|week|day}, baseline_type={average|absolute_min}, robust_max_percentile=95, R_percent=10

3. Shifting Without Spiking
    * Constraint: prevent shifting too many customers into the same low-MEF slot if it creates a regional peak.
    * Justification: Avoids all of the power being shifted to one time slot, causing potential overloads and instability in the grid.
    * Parameters Provided: alpha_peak_cap_percent=25 (per-slot upper cap: no more than +α% vs baseline)


**HARDCODED CONSTRAINTS:**

1. Total Consumption Conservation
    * Constraint: Σ_t x_t = Σ_t original_x_t for each customer.
    * Justification: The optimisation redistributes load; it cannot create or destroy energy.
    * Parameters Provided: id_field: customer_id, conservation_horizon = {day|week|month|year} (defaults to day)

2. Intra-Customer Conservation
    * Constraint: All shifts remain within the same customer ID.
    * Justification: No “borrowing” or “trading” energy between households.
    * Parameters Provided: id_field: customer_id

**Model Personality Levers**

1. Convenience Prioritised Model:
    * Shift the smallest total kWh possible while maximising emissions reduction per kWh shifted.
    * Likely higher customer adoption
    * Model: ConveniencePrioritised

2. Impact Prioritised Model:
    * Shift largest possible load from high-MEF to low-MEF within constraints.
    * Maximises emissions reduction, but at higher inconvenience.
    * Model: ImpactPrioritised

3. High-Usage-Focus Model
    * Target top percentile of consumers by total daily/weekly kWh.
    * Potentially fewer customers affected for same emissions gain.
    * Parameters Provided: target_high_usage_customers: True

| **#**  | **Constraint Name**              | **Description**                                                        | **Parameters Provided**                                                              | **Notes / Implementation**                                                            |                                                                                          |                                 |                                                                    |                                                                                                                       |
| ------ | -------------------------------- | ---------------------------------------------------------------------- | ------------------------------------------------------------------------------------ | ------------------------------------------------------------------------------------- | ---------------------------------------------------------------------------------------- | ------------------------------- | ------------------------------------------------------------------ | --------------------------------------------------------------------------------------------------------------------- |
| **B1** | Max Moves per Customer per Day   | Limit number of slot shifts per customer in a day to avoid disruption. | `X`, `timezone`, `day_boundaries` (default 0–23), `week_boundaries` (Mon–Sun or ISO) | Count moves as slots with any nonzero shift; day boundaries in `Asia/Kolkata`.        |                                                                                          |                                 |                                                                    |                                                                                                                       |
| **B2** | Max Moves per Customer per Week  | Limit number of shifts per week to prevent alert fatigue.              | `Y`, `timezone`, `week_boundaries` (Mon–Sun or ISO)                                  | Same counting method as daily; week can be fixed or ISO definition.                   |                                                                                          |                                 |                                                                    |                                                                                                                       |
| **B3** | Max Shift Window                 | Restrict shifts to within H hours of original slot.                    | `H_hours`, `slot_length=30min`, `inclusive=True`                                     | Apply as a mask to disallow moves beyond H hours; ensures realism.                    |                                                                                          |                                 |                                                                    |                                                                                                                       |
| **B4** | Peak-Hour Comfort                | Limit % reduction in peak hours to preserve convenience.               | `Z_percent`, \`scope={per\_customer                                                  | per\_city}`, `peak\_hours={city{Delhi,Mumbai},day,slots}\`                            | Cap is total % reduction across all listed hours; peak hours list provided per city/day. |                                 |                                                                    |                                                                                                                       |
| **T1** | Regional Maximum Shift %         | Cap total daily shifted load per city as % of average daily load.      | `P`, `regional_total_daily_average_load` (kWh), `city`                               | Apply per city; baseline from gov stats or calculated; prevents large system changes. |                                                                                          |                                 |                                                                    |                                                                                                                       |
| **T2** | Household Minimum Usage per Slot | Keep usage above baseline or % of robust max to maintain essentials.   | \`baseline\_period={year                                                             | month                                                                                 | week                                                                                     | day}`, `baseline\_type={average | absolute\_min}`, `robust\_max\_percentile=95`, `R\_percent=10\`    | `robust_max` = percentile max per customer + time bucket; floor = max(baseline, R% \* robust\_max); exclude outliers. |
| **T3** | Shifting Without Spiking         | Prevent overloading a low-MEF slot by shifting too much in.            | `alpha_peak_cap_percent=25`                                                          | Compare post-shift load to same-day baseline or historical slot avg; per city.        |                                                                                          |                                 |                                                                    |                                                                                                                       |
| **H1** | Total Consumption Conservation   | Preserve total consumption per customer over horizon.                  | `id_field=customer_id`, \`conservation\_horizon={day                                 | week                                                                                  | month                                                                                    | year}\` (default day)           | Sum of post-shift load must equal sum of original; tolerance 1e−6. |                                                                                                                       |
| **H2** | Intra-Customer Conservation      | All shifts must stay within the same household.                        | `id_field=customer_id`                                                               | No inter-household energy transfer allowed.                                           |                                                                                          |                                 |                                                                    |                                                                                                                       |


| **Model**                  | **Description**                                                              | **Parameters**                                             |
| -------------------------- | ---------------------------------------------------------------------------- | ---------------------------------------------------------- |
| **ConveniencePrioritised** | Minimise total shifted kWh while maximising emissions reduction/kWh shifted. | `min_total_shift_kWh` (optional soft cap)                  |
| **ImpactPrioritised**      | Maximise emissions reduction with largest possible shift within constraints. | `max_total_shift_kWh` (optional)                           |
| **HighUsageFocus**         | Target top percentile by kWh use to concentrate shifts.                      | `target_high_usage_customers=True`, `percentile_threshold` |


The user should easily be able to provide all of these parameters.
There should be multiple models that are used (different solvers ,etc.) -> IU'm thinking three families of solvers (MILP , Greedy, and then Lp, but within those probably specifications for their different solvers - such as highs, GLPK etc. - the user should also be able to specify these parameters.
There should be an option to show projgress to the user.
The solver should have an option True/Flase for run with parallelisation yes or no, and then a paralellisation method specification - i.e. mpi if using a distributed environment like HPC, or other for local paralellisation., or None if false.

Specialy notes:

the emission factors are in grams per kWh - I'm not sure if any additional calulatons / 2 etc are need since I have the half hour data.

The results should output all recommended shifts - listing the customer - the day , original time of usage, shifted (new) time, original emissions, emissions after shifting,delta in emissions, delta in power consumption, how many minutes forward or backward was moved, direction of the move, customer id

Optimises using MEF only.

Calculates both marginal savings and average-emissions savings for reporting.

Outputs:

“Marginal impact” (tCO₂ avoided)

“% reduction of marginal emissions”

“% reduction of total (average) emissions footprint”

### Importing Libraries

In [23]:
# ────────────────────────────────────────────────────────────────────────────
# Future (must be first)
# ────────────────────────────────────────────────────────────────────────────
from __future__ import annotations
import pyomo.environ as pyo

import numpy as np
from scipy.optimize import minimize


# ────────────────────────────────────────────────────────────────────────────
# Jupyter/Notebook Setup
# ────────────────────────────────────────────────────────────────────────────
%matplotlib inline
from IPython.display import display

# ────────────────────────────────────────────────────────────────────────────
# Standard Library
# ────────────────────────────────────────────────────────────────────────────
import binascii
import calendar
import json
import logging
import math
import os
import random
import re
import hashlib
import inspect
import time
from copy import deepcopy
from contextlib import contextmanager
from dataclasses import dataclass
from datetime import datetime, timedelta
from functools import partial, wraps
from itertools import combinations, product
from multiprocessing import Manager, Pool, Lock, cpu_count
from multiprocessing.pool import ThreadPool
from pathlib import Path
from typing import (
    Any, Callable, Dict, Iterable, List, Mapping, Optional, Sequence, Tuple, Union
)
from zoneinfo import ZoneInfo
import cvxpy as cp



# ────────────────────────────────────────────────────────────────────────────
# Core Data Handling
# ────────────────────────────────────────────────────────────────────────────
import numpy as np
import pandas as pd
import polars as pl

# ────────────────────────────────────────────────────────────────────────────
# Machine Learning & Statistics
# ────────────────────────────────────────────────────────────────────────────
from feature_engine.creation import CyclicalFeatures
from scipy.stats import kurtosis, skew, zscore
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.compose import ColumnTransformer
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    root_mean_squared_error,
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, SplineTransformer
from sklearn.utils.validation import check_is_fitted

# ────────────────────────────────────────────────────────────────────────────
# Visualization
# ────────────────────────────────────────────────────────────────────────────
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import seaborn as sns

# ────────────────────────────────────────────────────────────────────────────
# Geospatial
# ────────────────────────────────────────────────────────────────────────────
import geopandas as gpd
from shapely.geometry import Point, Polygon
from shapely.wkb import loads
from pyproj import Proj, transform


from dataclasses import dataclass
from typing import Optional


## Loading Data

### Directories

In [24]:
# DIRECTORIES AND PATHS
base_data_directory = "data"    # Base directory where the dataframes will be saved
hitachi_data_directory = os.path.join(base_data_directory, "hitachi_copy")      # Directory where the dataframes will be saved
meter_save_directory = os.path.join(hitachi_data_directory, "meter_primary_files")       # Directory for meter readings

marginal_emissions_development_directory = os.path.join(base_data_directory, "marginal_emissions_development")  # Directory for marginal emissions development data
marginal_emissions_results_directory = os.path.join(marginal_emissions_development_directory, "results")
marginal_emissions_logs_directory = os.path.join(marginal_emissions_development_directory, "logs")

optimisation_development_directory = os.path.join(base_data_directory, "optimisation_development")

In [25]:
print("\n" + "-" * 120)
print(f"Contents of '{optimisation_development_directory}' and subdirectories:\n" + "-" * 120)
for root, dirs, files in os.walk(optimisation_development_directory):
    for f in sorted(files):
        rel_dir = os.path.relpath(root, hitachi_data_directory)
        rel_file = os.path.join(rel_dir, f) if rel_dir != "." else f
        print(f"  - {rel_file}")


------------------------------------------------------------------------------------------------------------------------
Contents of 'data/optimisation_development' and subdirectories:
------------------------------------------------------------------------------------------------------------------------
  - ../optimisation_development/.DS_Store
  - ../optimisation_development/average_emissions_2022-05-04_to_2022-05-18.parquet
  - ../optimisation_development/customers_ids_with_emissions.parquet
  - ../optimisation_development/customers_ids_with_marginal_emissions.parquet
  - ../optimisation_development/marginal_and_average_emissions_2022-05-04_to_2022-05-18.parquet
  - ../optimisation_development/marginal_emissions_2022-05-04_to_2022-05-18.parquet
  - ../optimisation_development/meter_readings_2022-05-04_to_2022-05-18.parquet
  - ../optimisation_development/meter_readings_2022-05-04_to_2022-05-18_with_marginal_emissions.parquet
  - ../optimisation_development/results/.DS_Store


### File Paths

In [26]:
# Defining Full files
marginal_emissions_filename = "meter_readings_2022-05-04_to_2022-05-18_with_marginal_emissions"
marginal_emissions_filepath = os.path.join(optimisation_development_directory, marginal_emissions_filename + ".parquet")


In [27]:
marginal_emissions_pldf = pl.read_parquet(marginal_emissions_filepath)

In [28]:
print("-"  *120)
print("Schema of marginal_emissions_pldf\n"+ "-"  *120)
display(marginal_emissions_pldf.schema)

------------------------------------------------------------------------------------------------------------------------
Schema of marginal_emissions_pldf
------------------------------------------------------------------------------------------------------------------------


Schema([('ca_id', String),
        ('date', Datetime(time_unit='us', time_zone='Asia/Kolkata')),
        ('city', Categorical(ordering='physical')),
        ('customer_longitude', Float64),
        ('customer_latitude', Float64),
        ('value', Float64),
        ('demand_met_kWh', Float64),
        ('marginal_emissions_grams_co2_per_kWh', Float64),
        ('average_emissions_grams_co2_per_kWh', Float64)])

## Developing Optimisation Models

### Helper Functions

In [29]:
import polars as pl

def add_day_slot(df: pl.DataFrame) -> pl.DataFrame:
    # date already tz-aware Asia/Kolkata per your schema
    return (
        df
        .with_columns([
            pl.col("date").dt.truncate("1d").alias("day"),
            (pl.col("date").dt.hour() * 2 + (pl.col("date").dt.minute() >= 30).cast(pl.Int32)).alias("slot")
        ])
        .select(["ca_id", "city", "day", "date", "slot", "value", "ME", "customer_longitude", "customer_latitude"])
    )


In [30]:
import numpy as np

def group_to_arrays(sub: pl.DataFrame):
    # sub has columns: ['slot','value','ME', ...] for a single (ca_id, day)
    # Build 48-slot arrays, default zeros for usage; ME must be present to solve.
    usage = np.zeros(48, dtype=float)
    mef   = np.full(48, np.nan, dtype=float)

    sl = sub["slot"].to_numpy().astype(int)
    usage[sl] = sub["value"].to_numpy()
    mef[sl]   = sub["ME"].to_numpy()

    # If any ME is missing, you can either skip this day or impute.
    if np.isnan(mef).any():
        return None, None  # signal to skip
    return mef, usage


In [31]:
from datetime import timedelta

def materialise_optimised_rows(ca_id, city, day_ts, usage_opt, tz="Asia/Kolkata"):
    # day_ts is midnight (IST) for that day
    rows = []
    for s in range(48):
        ts = day_ts + timedelta(minutes=30*s)  # still tz-aware if day_ts is tz-aware
        rows.append({
            "ca_id": ca_id,
            "city": city,
            "date": ts,
            "day": day_ts,
            "slot": s,
            "optimised_value": float(usage_opt[s]),
        })
    return rows


In [32]:
# Window conversion
def hours_to_slots(window_hours: float) -> int:
    # half-hour data → each slot = 0.5 hour
    return int((window_hours * 60) // 30)

In [33]:
# Build allowed pairs within a window
def allowed_pairs(T: int, W_slots: int):
    # returns list of (t, s) with |t-s| <= W_slots
    pairs = []
    for t in range(T):
        s0 = max(0, t - W_slots)
        s1 = min(T, t + W_slots + 1)
        for s in range(s0, s1):
            pairs.append((t, s))
    return pairs

In [34]:
# Quick day-level pre-check (optional): upper bound of savings if you could freely reorder within window
def potential_savings_upper_bound(mef, usage, W_slots):
    # Conservative, fast heuristic: sort within window bands
    # (For a true bound you’d solve the continuous LP on that day; this is enough to gate tiny days.)
    # Return an estimated ΔE_upper (non-negative).
    # For initial runs, just return a large number to effectively disable the trigger.
    return float('inf')


In [35]:
@dataclass
class ShiftPolicy:
    window_hours: float                # e.g., 1.5
    min_day_trigger_frac: Optional[float] = None   # e.g., 0.05 → skip days with very low potential
    max_shift_frac: Optional[float] = None

#### Metrics 

In [36]:
def compute_metrics(mef, usage_base, usage_opt, flows, slot_minutes=30):
    mef = np.asarray(mef, float)
    u0  = np.asarray(usage_base, float)
    u1  = np.asarray(usage_opt,  float)

    E_base = float((u0 * mef).sum())
    E_opt  = float((u1 * mef).sum())
    dE     = E_base - E_opt

    moved_total = float(np.maximum(0, u0 - u1).sum())
    day_demand  = float(u0.sum())
    share_moved = moved_total / day_demand if day_demand > 0 else 0.0

    forward = sum(val for (t, s, val) in flows if s > t)
    backward = sum(val for (t, s, val) in flows if s < t)

    # avg shift distance in slots (and minutes)
    if flows:
        avg_steps = sum(abs(s - t) * val for (t, s, val) in flows) / moved_total
    else:
        avg_steps = 0.0
    avg_minutes = avg_steps * slot_minutes

    return {
        "E_base": E_base,
        "E_opt": E_opt,
        "delta_E": dE,
        "moved_kWh": moved_total,
        "share_moved": share_moved,
        "forward_kWh": forward,
        "backward_kWh": backward,
        "avg_shift_steps": avg_steps,
        "avg_shift_minutes": avg_minutes,
    }


#### Mixed Integer Linear Programming (MILP)

In [37]:
def optimise_day_milp(mef, usage, policy: ShiftPolicy):
    T = len(mef)
    W = hours_to_slots(policy.window_hours)
    pairs = allowed_pairs(T, W)
    day_demand = float(sum(usage))

    # Optional: skip day by trigger
    # if policy.min_day_trigger_frac:
    #   if potential_savings_upper_bound(...) < policy.min_day_trigger_frac * day_demand: return usage.copy(), None

    m = pyo.ConcreteModel()
    m.P = pyo.Set(initialize=list(range(len(pairs))))
    idx = {i: pairs[i] for i in range(len(pairs))}

    # Decision: flow from t to s
    m.y = pyo.Var(m.P, domain=pyo.NonNegativeReals)

    # Cost = emissions at destination s
    def obj_rule(m):
        return sum(m.y[i] * mef[idx[i][1]] for i in m.P)
    m.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

    # Every original slot’s energy must be fully re-assigned
    # sum_s y_{t->s} = usage_t
    def supply_rule(m, t):
        return sum(m.y[i] for i in m.P if idx[i][0] == t) == usage[t]
    m.supply = pyo.Constraint(range(T), rule=supply_rule)

    # (Optional) Max total shift cap per day:
    # Let "stay" be y_{t->t}. Then moved = day_demand - sum_t y_{t->t} <= max_shift_frac * day_demand
    if policy.max_shift_frac is not None:
        def cap_rule(m):
            stay = sum(m.y[i] for i in m.P if idx[i][0] == idx[i][1])
            return day_demand - stay <= policy.max_shift_frac * day_demand
        m.cap = pyo.Constraint(rule=cap_rule)

    # Solve
    solver = pyo.SolverFactory("cbc")  # or gurobi/cplex if available
    solver.solve(m, tee=False)

    # Reconstruct optimised usage and flows
    y = [pyo.value(m.y[i]) for i in m.P]
    usage_opt = [0.0] * T
    for i, val in enumerate(y):
        t, s = idx[i]
        usage_opt[s] += val

    # Optional: return the sparse flow list for metrics
    flows = [(idx[i][0], idx[i][1], y[i]) for i in m.P if y[i] > 1e-9]
    return usage_opt, flows


#### Continuous Optimisation 

In [38]:
def optimise_continuous(mef, usage, shift_window=2):
    """
    mef, usage: arrays shape (T,)
    shift_window: half-hour slots allowed for shifting
    """
    T = len(mef)

    def objective(x):
        return np.dot(x, mef)

    def constraint_total(x):
        return np.sum(x) - np.sum(usage)

    bounds = [(0, None) for _ in range(T)]

    # Initial guess = original usage
    x0 = usage.copy()

    cons = [{'type': 'eq', 'fun': constraint_total}]
    # Shift window constraint would need to be handled with penalty terms

    res = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=cons)
    return res.x

In [39]:
def optimise_day_lp(mef, usage, policy: ShiftPolicy):
    T = len(mef)
    W = hours_to_slots(policy.window_hours)
    pairs = allowed_pairs(T, W)
    P = len(pairs)
    day_demand = float(sum(usage))

    y = cp.Variable(P, nonneg=True)
    cost = cp.sum([y[i] * mef[pairs[i][1]] for i in range(P)])

    cons = []
    # Supply constraints: sum_s y_{t->s} = usage_t
    for t in range(T):
        cons.append(cp.sum([y[i] for i in range(P) if pairs[i][0] == t]) == usage[t])

    # Max shift cap (optional)
    if policy.max_shift_frac is not None:
        stay = cp.sum([y[i] for i in range(P) if pairs[i][0] == pairs[i][1]])
        cons.append(day_demand - stay <= policy.max_shift_frac * day_demand)

    prob = cp.Problem(cp.Minimize(cost), cons)
    prob.solve(solver=cp.GLPK)  # or ECOS/OSQP/CPLEX/Gurobi

    usage_opt = [0.0] * T
    for i, val in enumerate(y.value):
        t, s = pairs[i]
        usage_opt[s] += float(val)

    flows = [(pairs[i][0], pairs[i][1], float(y.value[i])) for i in range(P) if y.value[i] > 1e-9]
    return usage_opt, flows


#### Greedy Heuristic Skeleton


In [40]:
def greedy_shift(mef, usage, shift_window=2):
    """
    Simple greedy shift:
    Move load from highest-MEF slots to lowest-MEF slots within ±shift_window.
    """
    usage = usage.copy()
    T = len(usage)

    for t in np.argsort(-mef):  # high to low MEF
        if usage[t] > 0:
            window_start = max(0, t - shift_window)
            window_end = min(T, t + shift_window + 1)

            # Find best slot in window
            best_slot = min(range(window_start, window_end), key=lambda s: mef[s])
            if mef[best_slot] < mef[t]:
                shift_amount = usage[t]  # move all (can adapt to partial moves)
                usage[t] -= shift_amount
                usage[best_slot] += shift_amount
    return usage

In [41]:
def optimise_day_greedy(mef, usage, policy: ShiftPolicy):
    T = len(mef)
    W = hours_to_slots(policy.window_hours)
    usage_opt = usage.astype(float).copy()
    day_demand = float(usage_opt.sum())
    moved_so_far = 0.0
    flows = []

    # Optional: early exit via trigger (skip tiny days)
    # if policy.min_day_trigger_frac and day_demand < small_threshold: return usage_opt, []

    order = np.argsort(-mef)  # start from dirtiest slots
    for t in order:
        if usage_opt[t] <= 0:
            continue
        s0, s1 = max(0, t - W), min(T, t + W + 1)

        # pick the cleanest slot in the window
        s_best = min(range(s0, s1), key=lambda s: mef[s])
        if mef[s_best] >= mef[t]:
            continue  # no gain

        # How much can we move?
        can_move = usage_opt[t]

        # Respect max shift cap (optional)
        if policy.max_shift_frac is not None:
            cap_remaining = policy.max_shift_frac * day_demand - moved_so_far
            if cap_remaining <= 1e-9:
                break
            can_move = min(can_move, cap_remaining)

        # Respect optional min move trigger (slot-level)
        if policy.min_day_trigger_frac is not None:
            if can_move < policy.min_day_trigger_frac * day_demand:
                continue

        # Move the energy
        usage_opt[t] -= can_move
        usage_opt[s_best] += can_move
        moved_so_far += can_move
        flows.append((t, s_best, can_move))

    return usage_opt, flows


#### Orchestrator

In [42]:
from typing import Literal, Optional, Tuple, List, Dict

def run_optimisation_pipeline(
    df_raw: pl.DataFrame,
    policy: ShiftPolicy,
    method: Literal["lp","milp","greedy"]="lp",
    emit_optimised_rows: bool=False,
) -> Tuple[pl.DataFrame, pl.DataFrame, Optional[pl.DataFrame]]:

    df = add_day_slot(df_raw)

    # Ensure deterministic order for group iteration
    df = df.sort(["ca_id", "day", "slot"])

    metrics_rows: List[Dict] = []
    flow_rows: List[Dict] = []
    optim_rows: List[Dict] = [] if emit_optimised_rows else None

    # Group by (ca_id, day)
    gb = df.group_by(["ca_id", "day", "city"], maintain_order=True)

    # Pick solver function
    if method == "lp":
        solve_fn = optimise_day_lp
    elif method == "milp":
        solve_fn = optimise_day_milp
    elif method == "greedy":
        solve_fn = optimise_day_greedy
    else:
        raise ValueError("method must be one of: 'lp', 'milp', 'greedy'")

    for sub in gb:
        subdf = sub[1]  # (key, frame). Polars returns tuples when iterating groups.
        ca_id = subdf["ca_id"][0]
        city  = subdf["city"][0]
        day   = subdf["day"][0]

        mef, usage = group_to_arrays(subdf.select(["slot","value","ME"]))
        if mef is None:
            # Skip days with missing ME (or implement imputation)
            continue

        # Solve
        usage_opt, flows = solve_fn(mef, usage, policy)

        # Metrics
        m = compute_metrics(mef, usage, usage_opt, flows)
        metrics_rows.append({
            "ca_id": ca_id,
            "city": city,
            "day": day,
            **m
        })

        # Flows (t->s)
        for (t, s, val) in flows:
            flow_rows.append({
                "ca_id": ca_id,
                "city": city,
                "day": day,
                "t_slot": int(t),
                "s_slot": int(s),
                "kwh": float(val),
                "direction": "forward" if s > t else ("backward" if s < t else "stay"),
                "abs_steps": int(abs(s - t)),
            })

        # Optional: materialise optimised 48 rows
        if emit_optimised_rows:
            optim_rows.extend(materialise_optimised_rows(ca_id, city, day, usage_opt))

    metrics_df = pl.DataFrame(metrics_rows) if metrics_rows else pl.DataFrame()
    flows_df   = pl.DataFrame(flow_rows)   if flow_rows   else pl.DataFrame()
    optim_df   = pl.DataFrame(optim_rows)  if emit_optimised_rows and optim_rows else None

    return metrics_df, flows_df, optim_df


In [43]:
# daily_avg = df.group_by("ca_id").agg(pl.mean("value").alias("daily_avg"))
# weekly_avg = df.group_by(["ca_id", "week"]).agg(pl.mean("value").alias("weekly_avg"))


In [44]:
scenarios = [
    ShiftPolicy(window_hours=1, max_shift_frac=0.3),
    ShiftPolicy(window_hours=2, max_shift_frac=None),
    ShiftPolicy(window_hours=4, min_day_trigger_frac=0.05),
]
results = []
for policy in scenarios:
    metrics, flows, _ = run_optimisation_pipeline(df, policy, method="lp")
    metrics = metrics.with_columns(pl.lit(str(policy)).alias("policy"))
    results.append(metrics)
final_metrics = pl.concat(results)


NameError: name 'df' is not defined

### Implementation

#### Mixed Integer Programming

In [ ]:
import glpk
import cvxpy as cp
from cvxpy import GLPK

In [ ]:
# Example policy: 1.5h window, no trigger, no cap (baseline)
policy = ShiftPolicy(window_hours=1.5, min_day_trigger_frac=None, max_shift_frac=None)

metrics_df, flows_df, optim_df = run_optimisation_pipeline(
    df_raw=marginal_emissions_pldf,   # your joined table with ME + value
    policy=policy,
    method="lp",
    emit_optimised_rows=True
)

# Save for analysis
# metrics: one row per customer-day with emissions & movement stats
metrics_df.write_parquet("metrics_by_day.parquet")
# flows: sparse flows per customer-day, great for distance/direction analysis
flows_df.write_parquet("flows_by_day.parquet")
# optimised rows: 48 rows per customer-day with optimised_value
if optim_df is not None:
    optim_df.write_parquet("optimised_usage_rows.parquet")


In [ ]:
policy = ShiftPolicy(window_hours=1.0, max_shift_frac=0.3)  # cap at 30% of day demand
metrics_milp, flows_milp, _ = run_optimisation_pipeline(marginal_emissions_pldf, policy, method="milp", emit_optimised_rows=False)

NameError: name 'ShiftPolicy' is not defined

In [ ]:
policy = ShiftPolicy(window_hours=1.5, min_day_trigger_frac=None, max_shift_frac=0.30)
metrics_greedy, flows_greedy, _ = run_optimisation_pipeline(df_raw=marginal_emissions_pldf, policy, method="greedy")


In [ ]:
metrics_greedy

ca_id,city,day,E_base,E_opt,delta_E,moved_kWh,share_moved,forward_kWh,backward_kWh,avg_shift_steps,avg_shift_minutes
str,str,"datetime[μs, Asia/Kolkata]",f64,f64,f64,f64,f64,f64,f64,f64,f64
"""60000005516""","""delhi""",2022-05-03 00:00:00 IST,117.316896,113.919348,3.397549,89.4018,0.276361,45.6288,51.42,2.498187,74.945605
"""60000005516""","""delhi""",2022-05-07 00:00:00 IST,160.976491,155.999636,4.976855,113.246,0.259375,47.952,83.0313,3.058557,91.756698
"""60000005516""","""delhi""",2022-05-08 00:00:00 IST,153.855828,149.593574,4.262254,98.9624,0.246268,49.6704,70.884,3.201662,96.049873
"""60000005516""","""delhi""",2022-05-09 00:00:00 IST,163.711032,157.587469,6.123563,117.948,0.263893,68.7002,65.386,3.090698,92.720928
"""60000005516""","""delhi""",2022-05-10 00:00:00 IST,158.452079,152.711096,5.740984,109.1879,0.2521,22.924,107.0099,2.815476,84.464277
…,…,…,…,…,…,…,…,…,…,…,…
"""60029920067""","""delhi""",2022-05-09 00:00:00 IST,16.247836,15.879055,0.368782,12.897,0.3,6.307,6.59,2.428239,72.847174
"""60029920067""","""delhi""",2022-05-10 00:00:00 IST,16.314472,16.034948,0.279524,11.95,0.267398,8.717,4.69,2.557908,76.737238
"""60029920067""","""delhi""",2022-05-11 00:00:00 IST,20.166763,19.920712,0.246051,13.87,0.259349,5.184,10.86,2.711752,81.352559


In [ ]:
flows_greedy

ca_id,city,day,t_slot,s_slot,kwh,direction,abs_steps
str,str,"datetime[μs, Asia/Kolkata]",i64,i64,f64,str,i64
"""60000005516""","""delhi""",2022-05-03 00:00:00 IST,32,30,13.0,"""backward""",2
"""60000005516""","""delhi""",2022-05-03 00:00:00 IST,31,30,15.313,"""backward""",1
"""60000005516""","""delhi""",2022-05-03 00:00:00 IST,43,46,0.002,"""forward""",3
"""60000005516""","""delhi""",2022-05-03 00:00:00 IST,42,40,0.001,"""backward""",2
"""60000005516""","""delhi""",2022-05-03 00:00:00 IST,44,46,0.001,"""forward""",2
…,…,…,…,…,…,…,…
"""60029920067""","""delhi""",2022-05-13 00:00:00 IST,34,31,1.41,"""backward""",3
"""60029920067""","""delhi""",2022-05-13 00:00:00 IST,44,46,1.68,"""forward""",2
"""60029920067""","""delhi""",2022-05-13 00:00:00 IST,43,46,1.4,"""forward""",3


In [ ]:
# Save for analysis
# metrics: one row per customer-day with emissions & movement stats
metrics_df.write_parquet("metrics_by_day.parquet")
# flows: sparse flows per customer-day, great for distance/direction analysis
flows_df.write_parquet("flows_by_day.parquet")
# optimised rows: 48 rows per customer-day with optimised_value
if optim_df is not None:
    optim_df.write_parquet("optimised_usage_rows.parquet")

In [ ]:
# Emissions saved by customer over the full period
by_customer = (
    metrics_df
    .group_by("ca_id")
    .agg([
        pl.sum("delta_E").alias("total_delta_E"),
        pl.mean("share_moved").alias("avg_share_moved"),
        pl.count().alias("num_days"),
    ])
)

# Emissions saved by city and month
by_city_month = (
    metrics_df
    .with_columns(pl.col("day").dt.strftime("%Y-%m").alias("ym"))
    .group_by(["city", "ym"])
    .agg([
        pl.sum("delta_E").alias("delta_E_sum"),
        pl.mean("share_moved").alias("share_moved_mean"),
        pl.mean("avg_shift_minutes").alias("avg_shift_minutes_mean"),
    ])
)

# Direction & distance of moves (forward/backward, avg steps)
flow_summary = (
    flows_df
    .group_by(["city"])
    .agg([
        pl.sum(pl.when(pl.col("direction")=="forward").then(pl.col("kwh")).otherwise(0)).alias("forward_kWh"),
        pl.sum(pl.when(pl.col("direction")=="backward").then(pl.col("kwh")).otherwise(0)).alias("backward_kWh"),
        pl.mean("abs_steps").alias("avg_steps_weighted?"),  # careful: this isn't energy-weighted
    ])
)


In [ ]:
energy_weighted_steps = (
    flows_df
    .with_columns((pl.col("abs_steps") * pl.col("kwh")).alias("steps_x_kwh"))
    .group_by(["city"])
    .agg([
        (pl.sum("steps_x_kwh") / pl.sum("kwh")).alias("avg_steps_energy_weighted")
    ])
)


In [ ]:
try: prob.solve(solver=cp.GLPK)
except: prob.solve(solver=cp.ECOS)


1. Mixed Integer Linear Programming (MILP)
Why:

Exact, globally optimal solution under your constraints.

Handles discrete shifts (more realistic if you want to preserve original half-hour block structure).

Can easily encode “±2 hour” shift constraint via a binary feasibility matrix.

Pros:

Guarantees optimality.

Transparent formulation.

Easy to interpret results.

Cons:

Computationally heavy for long time horizons and many customers.

May need decomposition or batching for multi-year datasets.

Tooling:

PuLP, Pyomo, or ortools (Python) with solvers like CBC, Gurobi, CPLEX.

#### Quadratic / Constrained Linear Optimisation 

2. Quadratic / Constrained Linear Optimisation (Continuous Relaxation)
Why:

Fast if consumption is treated as divisible (continuous variables).

Works well if the real-world shifting flexibility is granular.

Can be solved with methods like Newton, Quasi-Newton, or Krylov subspace.

Pros:

Much faster than MILP.

Scales to multi-year data without decomposition.

Easily integrates smooth penalties (e.g., comfort penalties).

Cons:

May produce fractional solutions (not directly interpretable if you need discrete slots).

Not guaranteed to preserve "integer" block sizes without rounding.

Tooling:

scipy.optimize.minimize (SLSQP, trust-constr, Newton-CG, etc.),

cvxpy (continuous convex optimisation).

#### Greedy / Heuristic Shift Algorithm

3. Greedy / Heuristic Shift Algorithm
Why:

Computationally lightweight.

Good for very large datasets when exact MILP is impractical.

Works by iteratively moving load from highest-MEF slots to lowest-MEF available within ±2 hours.

Pros:

Very scalable.

Simple to implement and debug.

Produces feasible (though not guaranteed optimal) solutions.

Cons:

No optimality guarantee.

May get stuck in local minima.

Sensitive to tie-breaking rules.

Tooling:

Pure pandas/polars with NumPy-based operations.